# Step 3 - Points of interest based bicycle network generation
## Project: Growing Urban Bicycle Networks

This notebook follows the transit-oriented development approach of palominos2020ica or a grid approach and applies cardillo2006spp: Take the greedy triangulation between railway/underground stations (or other points of interest created in 02_prepare_pois). This is the cold start bicycle network generation process which creates bicycle networks from scratch.

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-06-18  
Last modified: 2021-01-23

## Preliminaries

### Parameters

In [1]:
debug = True # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"

Loaded parameters.



### Setup

In [2]:
%run -i path.py
%run -i setup.py

%load_ext watermark
%watermark -n -v -m -g -iv

Loaded PATH.



=== Cities ===
{   'newcastle': {   'countryid': 'gbr',
                     'name': 'Newcastle upon Tyne',
                     'nominatimstring': 'Newcastle upon Tyne'}}


Setup finished.

Python implementation: CPython
Python version       : 3.12.6
IPython version      : 8.29.0

Compiler    : MSC v.1941 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 140 Stepping 1, GenuineIntel
CPU cores   : 8
Architecture: 64bit

Git hash: 875cd764df6d00c25afeeae14b25e4036eecf8c9

geopandas : 0.14.4
haversine : 2.8.1
tqdm      : 4.66.5
networkx  : 3.3
shapely   : 2.0.6
osgeo     : 3.9.3
fiona     : 1.10.1
rasterio  : 1.3.11
owslib    : 0.32.0
geojson   : 3.1.0
numpy     : 1.26.4
json      : 2.0.9
watermark : 2.5.0
sys       : 3.12.6 | packaged by conda-forge | (main, Sep 22 2024, 14:01:26) [MSC v.1941 64 bit (AMD64)]
csv       : 1.0
igraph    : 0.11.6
matplotlib: 3.8.4
pandas    : 2.2.3
osmnx     : 1.9.4
IPython   : 8.2

### Functions

In [3]:
%run -i functions.py

Loaded functions.



## Routing (shortest paths)

The function and code below currently routes between the edges of neighbourhoods, rather than from a single point to a single point. We then join the neighbourhoods up first, before considering the wider area. This wider area is derived from hexagonal tesslleations within the city boundaries.

In [16]:
def greedy_triangulation_routing_mix(G, neighbourhood_nnids, tessellation_nnids, weighting=None, prune_quantiles=[1], prune_measure="betweenness"):
    import warnings
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=UserWarning)
    warnings.filterwarnings("ignore", category=DtypeWarning)

    # Combine neighbourhood_nnids and tessellation_nnids with origin labels
    pois = [(node_id, 'neighbourhood') for node_id in neighbourhood_nnids] + \
           [(node_id, 'tessellation') for node_id in tessellation_nnids]

    if len(pois) < 2: 
        return [], []  # We can't do anything with fewer than 2 POIs

    # Separate POIs by origin (neighbourhood and tessellation)
    neighbourhood_pois = [poi for poi in pois if poi[1] == 'neighbourhood']
    tessellation_pois = [poi for poi in pois if poi[1] == 'tessellation']

    # Extract node IDs for use in GT creation
    neighbourhood_ids = [poi[0] for poi in neighbourhood_pois]
    tessellation_ids = [poi[0] for poi in tessellation_pois]

    # Create POI pairs and indices for neighbourhoods first
    pois_indices = [G.vs.find(id=poi[0]).index for poi in neighbourhood_pois]
    poipairs = poipairs_by_distance(G, neighbourhood_ids, weighting, True)

    if len(poipairs) == 0: 
        return [], []

    # Handle edge order based on prune_measure
    # create a random order for edges
    if prune_measure == "random":
        GT = copy.deepcopy(G.subgraph(pois_indices))
        for poipair, poipair_distance in poipairs:
            poipair_ind = (GT.vs.find(id=poipair[0]).index, GT.vs.find(id=poipair[1]).index)
            if not new_edge_intersects(GT, (GT.vs[poipair_ind[0]]["x"], GT.vs[poipair_ind[0]]["y"],
                                            GT.vs[poipair_ind[1]]["x"], GT.vs[poipair_ind[1]]["y"])):
                GT.add_edge(poipair_ind[0], poipair_ind[1], weight=poipair_distance)
        random.seed(0)
        edgeorder = random.sample(range(GT.ecount()), k=GT.ecount())
    else:
        edgeorder = False

    # Initialisation
    GT_abstracts = []
    neighbourhood_GT_abstracts = []
    tessellation_GT_abstracts = []
    GTs = []
    all_shortest_paths = []
    processed_pairs = set()
    GT_indices = set()

    # Greedy triangulation for neighbourhood POIs first
    for prune_quantile in tqdm(prune_quantiles, desc="Greedy triangulation", leave=False):
        GT_abstract = copy.deepcopy(G.subgraph(pois_indices))
        GT_abstract = greedy_triangulation(GT_abstract, poipairs, prune_quantile, prune_measure, edgeorder)
        neighbourhood_GT_abstracts.append(GT_abstract)

        if debug:
            temp = ig_to_geojson(GT_abstract)
            temp_geo = [shape(geometry) for geometry in temp["geometries"]]
            temp_gdf = gpd.GeoDataFrame(geometry=temp_geo, crs="EPSG:4326")
            ax = temp_gdf.plot(color="red", label=f"GT_abstract for prune_quantile {prune_quantile}")
            all_centroids.plot(ax=ax, color="green", marker="o", markersize=50, label="Neighbourhood Source Point")
            plt.title(f"GT_abstract for prune_quantile {prune_quantile}")
            plt.show()

    # Process tessellation POIs
    pois_indices = [G.vs.find(id=poi[0]).index for poi in tessellation_pois]
    poipairs = poipairs_by_distance(G, tessellation_ids, weighting, True)

    for prune_quantile in tqdm(prune_quantiles, desc="Greedy triangulation for tessellation", leave=False):
        GT_abstract = copy.deepcopy(G.subgraph(pois_indices))
        GT_abstract = greedy_triangulation(GT_abstract, poipairs, prune_quantile, prune_measure, edgeorder)
        tessellation_GT_abstracts.append(GT_abstract)

        if debug:
            temp = ig_to_geojson(GT_abstract)
            temp_geo = [shape(geometry) for geometry in temp["geometries"]]
            temp_gdf2 = gpd.GeoDataFrame(geometry=temp_geo, crs="EPSG:4326")
            ax = temp_gdf2.plot()
            temp_gdf.plot(ax=ax, color="red")
            all_centroids.plot(ax=ax, color="green", marker="o", markersize=50, label="All Centroids")
            plt.title("Abstract for tessellation")
            plt.show()

    # Combine neighbourhood and tessellation graphs
    GT_abstracts = neighbourhood_GT_abstracts + tessellation_GT_abstracts

    # Assign source labels to edges in the combined abstracts
    for GT_abstract in GT_abstracts:
        source_value = "neighbourhoods" if GT_abstract in neighbourhood_GT_abstracts else "tessellation"
        for edge in GT_abstract.es:
            edge["source"] = source_value

    # Routing for GT_abstracts
    GT_indices = set()
    GT_abstracts_iterator = iter(GT_abstracts)

    ## dounle the number of times we iterate as we have two input graphs rather than one
    # note this may need changing
    def double_prune_quantiles(prune_quantiles):
        doubled_quantiles = []
        for i in range(len(prune_quantiles) - 1):
            doubled_quantiles.append(prune_quantiles[i])
            doubled_quantiles.append((prune_quantiles[i] + prune_quantiles[i + 1]) / 2)
        doubled_quantiles.append(prune_quantiles[-1])
        return doubled_quantiles
    dubs = double_prune_quantiles(prune_quantiles)

    # do the routing
    for prune_quantile in tqdm(dubs, desc="Routing", leave=False):
        GT_abstract = next(GT_abstracts_iterator)
        routenodepairs = {(e.source_vertex["id"], e.target_vertex["id"]): e["weight"] for e in GT_abstract.es}
        routenodepairs = sorted(routenodepairs.items(), key=lambda x: x[1])

        for poipair, poipair_distance in routenodepairs:
            poipair_ind = (G.vs.find(id=poipair[0]).index, G.vs.find(id=poipair[1]).index)
            
            # Neighbourhood routing logic
            if any(poi[0] == poipair[0] and poi[1] == 'neighbourhood' for poi in pois):
                neighbourhood_a = all_centroids.loc[all_centroids['nearest_node'] == poipair[0], 'neighbourhood_id'].values[0]
                neighbourhood_b = all_centroids.loc[all_centroids['nearest_node'] == poipair[1], 'neighbourhood_id'].values[0]
                exit_points_a = exit_points[exit_points['neighbourhood_id'] == neighbourhood_a].index
                exit_points_b = exit_points[exit_points['neighbourhood_id'] == neighbourhood_b].index

                shortest_path_length, best_path = float('inf'), None
                for ea in exit_points_a:
                    for eb in exit_points_b:
                        pair_id = tuple(sorted((ea, eb)))
                        if pair_id in processed_pairs: 
                            continue
                        processed_pairs.add(pair_id)
                        ea_vertex_index = G.vs.find(id=ea).index
                        eb_vertex_index = G.vs.find(id=eb).index
                        sp = G.get_shortest_paths(ea_vertex_index, eb_vertex_index, weights="weight", output="vpath")[0]
                        if sp and len(sp) < shortest_path_length:
                            shortest_path_length, best_path = len(sp), sp

                if best_path:
                    all_shortest_paths.append((poipair[0], poipair[1], shortest_path_length, best_path))
                    GT_indices.update(best_path)

                    GT = G.induced_subgraph([G.vs[idx] for idx in best_path])
                    GTs.append(GT)
            else:
                # Routing for tessellation POIs
                sp = set(G.get_shortest_paths(poipair_ind[0], poipair_ind[1], weights="weight", output="vpath")[0])
                GT_indices = GT_indices.union(sp)

        # Final GT creation
        GT = G.induced_subgraph(GT_indices)
        GTs.append(GT)

        if debug:
            GT_gjson = ig_to_geojson(GT)
            GT_geometries = [shape(geometry) for geometry in GT_gjson["geometries"]]
            GT_gdf = gpd.GeoDataFrame(geometry=GT_geometries, crs="EPSG:4326")
            fig, ax = plt.subplots(figsize=(10, 10))
            GT_gdf.plot(ax=ax, color="blue", linewidth=1.5, label=f"GT for prune_quantile {prune_quantile}")
            all_centroids.plot(ax=ax, color="green", marker="o", markersize=50, label="All Centroids")
            exit_points.plot(ax=ax, color="red", marker="x", markersize=30, label="All Exit Points")
            plt.legend()
            plt.xlabel("Longitude")
            plt.ylabel("Latitude")
            plt.title(f"Greedy Triangulation (GT) for prune_quantile = {prune_quantile}")
            plt.show()

    return GTs, GT_abstracts


In [19]:
## do the triangulation and routing

# Load data again to ensure we account for neighbourhoods correctly
# Load all carall graphs in OSMNX format
G_caralls = {}
G_caralls_simplified = {}
locations = {}
parameterinfo = osmnxparameters['carall']





for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):
    print(f"{placeid}: Loading location polygon and carall graph")
    
    if placeinfo["nominatimstring"] != '':
        location = ox.geocoder.geocode_to_gdf(placeinfo["nominatimstring"])
        if location.geometry[0].geom_type == 'MultiPolygon':
            location = location.explode(index_parts=False).reset_index(drop=True)
        location = fill_holes(extract_relevant_polygon(placeid, shapely.geometry.shape(location['geometry'][0])))
    else:
        # https://gis.stackexchange.com/questions/113799/how-to-read-a-shapefile-in-python
        shp = fiona.open(PATH["data"] + placeid + "/" + placeid + ".shp")
        first = next(iter(shp))
        try:
            location = Polygon(shapely.geometry.shape(first['geometry']))  # If shape file is given as linestring
        except:
            location = shapely.geometry.shape(first['geometry'])
    locations[placeid] = location
    
    G_caralls[placeid] = csv_to_ox(PATH["data"] + placeid + "/", placeid, 'carall')
    G_caralls[placeid].graph["crs"] = 'epsg:4326'  # Needed for OSMNX's graph_to_gdfs in utils_graph.py
    G_caralls_simplified[placeid] = csv_to_ox(PATH["data"] + placeid + "/", placeid, 'carall_simplified')
    G_caralls_simplified[placeid].graph["crs"] = 'epsg:4326'  # Needed for OSMNX's graph_to_gdfs in utils_graph.py

    print(f"{placeid}: Loading and moving POIs")
    # Get the carall graph and location geometry
    location = locations[placeid]
    G_carall = G_caralls_simplified[placeid]

    # Load neighbourhoods and create GeoDataFrame for centroids
    neighbourhoods = load_neighbourhoods(PATH["data"] + placeid + "/")
    all_centroids = gpd.GeoDataFrame(columns=['neighbourhood_id', 'geometry'], crs='EPSG:4326')  
    exit_points = get_exit_nodes(neighbourhoods, G_carall)  # Requires osmnx G_carall, not igraph G_carall
        
    unique_id = 0  # Counter for unique IDs across neighbourhoods

    for name, gdf in neighbourhoods.items():  # Process each neighbourhood GeoDataFrame to get centroids, exit points, and neighbourhood IDs
        if gdf.empty:
            print(f"Warning: The GeoDataFrame for {name} is empty. Skipping...")
            continue
        print(f"Processing neighbourhoods in: {name}")

        # Assign a unique ID to each neighbourhood in the GeoDataFrame to reference throughout
        gdf['neighbourhood_id'] = range(unique_id, unique_id + len(gdf))
        if debug:
            print(f"Assigned neighbourhood_ids from {unique_id} to {unique_id + len(gdf) - 1} for {name}")

        # Get centroids to inherit 'neighbourhood_id'
        centroids_gdf = get_neighbourhood_centroids(gdf)
        all_centroids = pd.concat([all_centroids, centroids_gdf], ignore_index=True)
        unique_id += len(gdf)  # Increment by the number of neighbourhoods processed

    # Snap centroids to the closest nodes in the street network
    neighbourhood_nnids = set()
    for g in all_centroids['geometry']:
        n = ox.distance.nearest_nodes(G_carall, g.x, g.y)
        if n not in neighbourhood_nnids and haversine((g.y, g.x), (G_carall.nodes[n]["y"], G_carall.nodes[n]["x"]), unit="m") <= snapthreshold:
            neighbourhood_nnids.add(n)
    # Add nearest_node column to all_centroids by finding the nearest node for each centroid geometry
    all_centroids['nearest_node'] = all_centroids['geometry'].apply(
        lambda g: ox.distance.nearest_nodes(G_carall, g.x, g.y))  # We now have all_centroids with 'neighbourhood_id', 'geometry', 'nearest_node' columns

    # Load POIs
    with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidsbikeall.csv') as f:
        tessellation_nnids = [int(line.rstrip()) for line in f]

    # Combine nnids
    nnids = neighbourhood_nnids | set(tessellation_nnids)

    # Load networks
    # G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall', weighting=weighting)
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrackcarall', weighting=weighting)  # To load car and cycle network

    # Generation
    (GTs, GT_abstracts) = greedy_triangulation_routing_mix(G_carall, neighbourhood_nnids, tessellation_nnids, weighting, prune_quantiles, prune_measure)
    (MST, MST_abstract) = mst_routing(G_carall, nnids, weighting)
    
    # Restore original edge lengths
    if weighting:
        restore_original_lengths(G_carall)
        for GT in GTs:
            restore_original_lengths(GT)
        restore_original_lengths(MST)

    # Write results
    results = {
        "placeid": placeid,
        "prune_measure": prune_measure,
        "poi_source": poi_source,
        "prune_quantiles": prune_quantiles,
        "GTs": GTs,
        "GT_abstracts": GT_abstracts,
        "MST": MST,
        "MST_abstract": MST_abstract
    }
    write_result(results, "pickle", placeid, poi_source, prune_measure, ".pickle", weighting=weighting)


Cities:   0%|          | 0/1 [00:00<?, ?it/s]

newcastle: Loading location polygon and carall graph
newcastle: Loading and moving POIs
1 Cities loaded
Processing neighbourhoods in: Newcastle Upon Tyne
Loaded neighbourhoods and exit points for newcastle: Generating networks
Loaded tessellation points


C:\Users\b8008458\OneDrive - Newcastle University\2022 to 2023\PhD\networkGrowth\bikenwgrowth\code\functions.py:448: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  e = pd.read_csv(p + prefix + '_edges.csv')


Greedy triangulation:   0%|          | 0/40 [00:00<?, ?it/s]

Greedy triangulation for tessellation:   0%|          | 0/40 [00:00<?, ?it/s]

Routing:   0%|          | 0/79 [00:00<?, ?it/s]

In [ ]:
break

### Testing below

In [124]:
def poipairs_by_distance_mix(G, pois, weighting=None, return_distances=False):
    """
    Calculates the (weighted) graph distances on G for a subset of nodes pois.
    Returns all pairs of POI ids sorted by:
    1. Source (neighbourhood-to-neighbourhood pairs first, then others).
    2. Distance (ascending order).
    If return_distances, then distances are also returned.
    """
    # Get poi indices
    indices = [G.vs.find(id=poi).index for poi in pois]

    # Map POI ids to their source type (neighbourhood or tessellation)
    poi_sources = {poi: 'neighbourhood' if poi in neighbourhood_nnids else 'tessellation' for poi in pois}

    # Get sequences of nodes and edges in shortest paths between all pairs of pois
    poi_nodes = []
    poi_edges = []
    for c, v in enumerate(indices):
        poi_nodes.append(G.get_shortest_paths(v, indices[c:], weights="weight", output="vpath"))
        poi_edges.append(G.get_shortest_paths(v, indices[c:], weights="weight", output="epath"))

    # Sum up weights (distances) of all paths
    poi_dist = {}
    for paths_n, paths_e in zip(poi_nodes, poi_edges):
        for path_n, path_e in zip(paths_n, paths_e):
            if weighting:
                path_dist = sum([G.es[e]['ori_length'] for e in path_e])  # Use 'ori_length' for distance
            else:
                path_dist = sum([G.es[e]['weight'] for e in path_e])  # Fallback to 'weight' if weighting is False

            if path_dist > 0:
                poi_dist[(path_n[0], path_n[-1])] = path_dist

    # Back to POI ids and add source information
    temp = [
        ((G.vs[p[0][0]]["id"], G.vs[p[0][1]]["id"]), p[1],
         poi_sources[G.vs[p[0][0]]["id"]], poi_sources[G.vs[p[0][1]]["id"]])
        for p in poi_dist.items()
    ]

    # Sort by source (neighbourhood-to-neighbourhood first, then others), then by distance
    temp_sorted = sorted(
        temp,
        key=lambda x: (
            0 if x[2] == 'neighbourhood' and x[3] == 'neighbourhood' else 1,  # Primary: neighbourhood-to-neighbourhood first
            x[1]  # Secondary: ascending distance
        )
    )

    # Remove source information before returning (optional)
    if return_distances:
        return [(p[0], p[1]) for p in temp_sorted]
    else:
        return [p[0] for p in temp_sorted]


In [131]:
#### this try is to mix both into the same poi pairs


def greedy_triangulation_routing_mix(G, neighbourhood_nnids, tessellation_nnids, weighting=None, prune_quantiles=[1], prune_measure="betweenness"):
    import warnings
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=UserWarning)

    # Combine neighbourhood_nnids and tessellation_nnids with origin labels
    pois = [(node_id, 'neighbourhood') for node_id in neighbourhood_nnids] + \
           [(node_id, 'tessellation') for node_id in tessellation_nnids]

    if len(pois) < 2: 
        return [], []  # We can't do anything with less than 2 POIs

    # Separate POIs by origin (neighbourhood and tessellation)
    neighbourhood_pois = [poi for poi in pois if poi[1] == 'neighbourhood']
    tessellation_pois = [poi for poi in pois if poi[1] == 'tessellation']

    # Extract only the node ids for use in GT creation
    neighbourhood_ids = [poi[0] for poi in neighbourhood_pois]
    tessellation_ids = [poi[0] for poi in tessellation_pois]

    # Combine all POI indices and create POI pairs for all POIs
    combined_ids = neighbourhood_ids + tessellation_ids
    pois_indices = [G.vs.find(id=poi).index for poi in combined_ids]
    G_temp = copy.deepcopy(G)
    for e in G_temp.es:  # Delete all edges
        G_temp.es.delete(e)

    # Use poipairs_by_distance for the combined set of points
    poipairs = poipairs_by_distance_mix(G, combined_ids, weighting, True)

    print(poipairs)


    if len(poipairs) == 0: 
        return [], []

    if prune_measure == "random":
        # Create a random order for edges
        GT = copy.deepcopy(G.subgraph(pois_indices))
        for poipair, poipair_distance in poipairs:
            poipair_ind = (GT.vs.find(id=poipair[0]).index, GT.vs.find(id=poipair[1]).index)
            if not new_edge_intersects(GT, (GT.vs[poipair_ind[0]]["x"], GT.vs[poipair_ind[0]]["y"], GT.vs[poipair_ind[1]]["x"], GT.vs[poipair_ind[1]]["y"])):
                GT.add_edge(poipair_ind[0], poipair_ind[1], weight=poipair_distance)
        random.seed(0)
        edgeorder = random.sample(range(GT.ecount()), k=GT.ecount())
    else:
        edgeorder = False

    GT_abstracts = []
    GTs = []
    all_shortest_paths = []
    processed_pairs = set()
    GT_indices = set()  # Accumulated nodes for the final GT

    # Greedy Triangulation for neighbourhood pois first
    for prune_quantile in tqdm(prune_quantiles, desc="Greedy triangulation", leave=False):
        # GT creation for "neighbourhood" pois first
        GT_abstract = copy.deepcopy(G.subgraph(pois_indices))
        GT_abstract = greedy_triangulation(GT_abstract, poipairs, prune_quantile, prune_measure, edgeorder)
        GT_abstracts.append(GT_abstract)
        ############ Temporary code for debugging purposes ############
        if debug:
            temp = ig_to_geojson(GT_abstract)
            temp_geo = [shape(geometry) for geometry in temp["geometries"]]
            temp_gdf = gpd.GeoDataFrame(geometry=temp_geo, crs="EPSG:4326")
            print("Neighbourhood pois plot")
            ax = temp_gdf.plot(color = "red", label=f"GT_abstract for prune_quantile {prune_quantile}")
            all_centroids.plot(ax=ax, color="green", marker="o", markersize=50, label="Neighbourhood Source Point")
            plt.title(f"GT_abstract for prune_quantile {prune_quantile}")
            plt.show()   
        ###############################################################     


    ############ Temporary code for debugging purposes ############
    if debug:
        temp = ig_to_geojson(GT_abstract)
        temp_geo = [shape(geometry) for geometry in temp["geometries"]]
        temp_gdf = gpd.GeoDataFrame(geometry=temp_geo, crs="EPSG:4326")
        print("Neighbourhood pois plot")
        ax = temp_gdf.plot(color = "red")
        all_centroids.plot(ax=ax, color="green", marker="o", markersize=50, label="Neighbourhood Source Point")
        plt.title("GT_Abstract for neighbourhoods")
        plt.show()   
    ###############################################################     


    
    if debug:
        for edge in GT_abstracts[0].es:
            print(edge.attributes())


    # Routing and creating GT for each prune_quantile
    GT_indices = set()

    # set an interator to do processing per GT_abstract in GT_abstracts
    for prune_quantile in tqdm(prune_quantiles, desc="Routing", leave=False):
        # Update GT_abstract within each prune_quantile
        GT_abstract = copy.deepcopy(G_temp.subgraph(pois_indices))
        GT_abstract = greedy_triangulation(GT_abstract, poipairs, prune_quantile, prune_measure, edgeorder)
        GT_abstracts.append(GT_abstract)
        
        # get node pairs we need to route, sorted by distance
        routenodepairs = { (e.source_vertex["id"], e.target_vertex["id"]): e["weight"] for e in GT_abstract.es }
        routenodepairs = sorted(routenodepairs.items(), key=lambda x: x[1])

        for poipair, poipair_distance in routenodepairs:
            poipair_ind = (G.vs.find(id=poipair[0]).index, G.vs.find(id=poipair[1]).index)
            
            if any(poi[0] == poipair[0] and poi[1] == 'neighbourhood' for poi in pois):
                # Neighbourhood routing logic
                neighbourhood_a = all_centroids.loc[all_centroids['nearest_node'] == poipair[0], 'neighbourhood_id'].values[0]
                neighbourhood_b = all_centroids.loc[all_centroids['nearest_node'] == poipair[1], 'neighbourhood_id'].values[0]
                exit_points_a = exit_points[exit_points['neighbourhood_id'] == neighbourhood_a].index
                exit_points_b = exit_points[exit_points['neighbourhood_id'] == neighbourhood_b].index

                shortest_path_length, best_path = float('inf'), None
                for ea in exit_points_a:
                    for eb in exit_points_b:
                        pair_id = tuple(sorted((ea, eb)))
                        if pair_id in processed_pairs: 
                            continue
                        processed_pairs.add(pair_id)
                        ea_vertex_index = G.vs.find(id=ea).index
                        eb_vertex_index = G.vs.find(id=eb).index
                        sp = G.get_shortest_paths(ea_vertex_index, eb_vertex_index, weights="weight", output="vpath")[0]
                        if sp and len(sp) < shortest_path_length:
                            shortest_path_length, best_path = len(sp), sp

                if best_path:
                    all_shortest_paths.append((poipair[0], poipair[1], shortest_path_length, best_path))
                    GT_indices.update(best_path)
                    
                    # Plotting for neighbourhood routing
                    GT = G.induced_subgraph([G.vs[idx] for idx in best_path])
                    GTs.append(GT)
            else:
                # Routing for tessellation pois (more general case)
                sp = set(G.get_shortest_paths(poipair_ind[0], poipair_ind[1], weights="weight", output="vpath")[0])
                GT_indices = GT_indices.union(sp)

            
        # Final step: Create the final GT from the indices we've accumulated
        GT = G.induced_subgraph(GT_indices)
        GTs.append(GT)

        # Plot the current GT
        GT_gjson = ig_to_geojson(GT)
        GT_geometries = [shape(geometry) for geometry in GT_gjson["geometries"]]
        GT_gdf = gpd.GeoDataFrame(geometry=GT_geometries, crs="EPSG:4326")

        # Plotting the GTs after each loop
        fig, ax = plt.subplots(figsize=(10, 10))
        GT_gdf.plot(ax=ax, color="blue", linewidth=1.5, label=f"GT for prune_quantile {prune_quantile}")
        
        all_centroids.plot(ax=ax, color="green", marker="o", markersize=50, label="All Centroids")
        exit_points.plot(ax=ax, color="red", marker="x", markersize=30, label="All Exit Points")

        plt.legend()
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.title(f"Greedy Triangulation (GT) for prune_quantile = {prune_quantile}")
        plt.show()

    return GTs, GT_abstracts


In [ ]:
### THIS FUNCTION WORKS FOR ONE OR THE OTHER AT AT TIME


def greedy_triangulation_routing_mix(G, neighbourhood_nnids, tessellation_nnids, weighting=None, prune_quantiles=[1], prune_measure="betweenness"):
    import warnings
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=UserWarning)

    # Combine neighbourhood_nnids and tessellation_nnids with origin labels
    pois = [(node_id, 'neighbourhood') for node_id in neighbourhood_nnids] + \
           [(node_id, 'tessellation') for node_id in tessellation_nnids]

    if len(pois) < 2: 
        return [], []  # We can't do anything with less than 2 POIs

    # Separate pois by origin (neighbourhood and tessellation)
    neighbourhood_pois = [poi for poi in pois if poi[1] == 'neighbourhood']
    tessellation_pois = [poi for poi in pois if poi[1] == 'tessellation']

    # Extract only the node ids for use in GT creation
    neighbourhood_ids = [poi[0] for poi in neighbourhood_pois]
    tessellation_ids = [poi[0] for poi in tessellation_pois]

    # Create POI pairs and indices for neighbourhoods first
    pois_indices = [G.vs.find(id=poi[0]).index for poi in neighbourhood_pois]
    poipairs = poipairs_by_distance(G, neighbourhood_ids, weighting, True)

    if len(poipairs) == 0: 
        return [], []

    if prune_measure == "random":
        # Create a random order for edges
        GT = copy.deepcopy(G.subgraph(pois_indices))
        for poipair, poipair_distance in poipairs:
            poipair_ind = (GT.vs.find(id=poipair[0]).index, GT.vs.find(id=poipair[1]).index)
            if not new_edge_intersects(GT, (GT.vs[poipair_ind[0]]["x"], GT.vs[poipair_ind[0]]["y"], GT.vs[poipair_ind[1]]["x"], GT.vs[poipair_ind[1]]["y"])):
                GT.add_edge(poipair_ind[0], poipair_ind[1], weight=poipair_distance)
        random.seed(0)
        edgeorder = random.sample(range(GT.ecount()), k=GT.ecount())
    else:
        edgeorder = False

    GT_abstracts = []
    neighbourhood_GT_abstracts = []
    tessellation_GT_abstracts = []
    GTs = []
    all_shortest_paths = []
    processed_pairs = set()
    GT_indices = set()  # Accumulated nodes for the final GT

    # Greedy Triangulation for neighbourhood pois first
    for prune_quantile in tqdm(prune_quantiles, desc="Greedy triangulation", leave=False):
        # GT creation for "neighbourhood" pois first
        GT_abstract = copy.deepcopy(G.subgraph(pois_indices))
        GT_abstract = greedy_triangulation(GT_abstract, poipairs, prune_quantile, prune_measure, edgeorder)
        neighbourhood_GT_abstracts.append(GT_abstract)
        ############ Temporary code for debugging purposes ############
        if debug:
            temp = ig_to_geojson(GT_abstract)
            temp_geo = [shape(geometry) for geometry in temp["geometries"]]
            temp_gdf = gpd.GeoDataFrame(geometry=temp_geo, crs="EPSG:4326")
            print("Neighbourhood pois plot")
            ax = temp_gdf.plot(color = "red", label=f"GT_abstract for prune_quantile {prune_quantile}")
            all_centroids.plot(ax=ax, color="green", marker="o", markersize=50, label="Neighbourhood Source Point")
            plt.title(f"GT_abstract for prune_quantile {prune_quantile}")
            plt.show()   
        ###############################################################     


    ############ Temporary code for debugging purposes ############
    if debug:
        temp = ig_to_geojson(GT_abstract)
        temp_geo = [shape(geometry) for geometry in temp["geometries"]]
        temp_gdf = gpd.GeoDataFrame(geometry=temp_geo, crs="EPSG:4326")
        print("Neighbourhood pois plot")
        ax = temp_gdf.plot(color = "red")
        all_centroids.plot(ax=ax, color="green", marker="o", markersize=50, label="Neighbourhood Source Point")
        plt.title("GT_Abstract for neighbourhoods")
        plt.show()   
    ###############################################################     


    # After processing all neighbourhood pois, now process tessellation pois
    pois_indices = [G.vs.find(id=poi[0]).index for poi in tessellation_pois]
    poipairs = poipairs_by_distance(G, tessellation_ids, weighting, True)

    # GT creation for "tessellation" pois
    for prune_quantile in tqdm(prune_quantiles, desc="Greedy triangulation for tessellation", leave=False):
        # Perform greedy triangulation for tessellation pois
        GT_abstract = copy.deepcopy(G.subgraph(pois_indices))
        GT_abstract = greedy_triangulation(GT_abstract, poipairs, prune_quantile, prune_measure, edgeorder)
        tessellation_GT_abstracts.append(GT_abstract)



    ############ Temporary code for debugging purposes ############
    if debug:
        temp = ig_to_geojson(GT_abstract)
        temp_geo = [shape(geometry) for geometry in temp["geometries"]]
        temp_gdf2 = gpd.GeoDataFrame(geometry=temp_geo, crs="EPSG:4326")
        ax = temp_gdf2.plot()
        temp_gdf.plot(ax=ax, color = "red")
        all_centroids.plot(ax=ax, color="green", marker="o", markersize=50, label="All Centroids")
        plt.title("Abstract for tesselation")
        plt.show()
    ###############################################################

    



    # note that this is two different abstract graphs mashed together, not one nice triangulated graph
    # this may need changing, I am using to test the routing section   
    GT_abstracts =  neighbourhood_GT_abstracts + tessellation_GT_abstracts 
    # to join two abstract graphs together, we ensure we can keep track of the source
    for i, GT_abstract in enumerate(GT_abstracts):
        # Determine the source type
        if GT_abstract in neighbourhood_GT_abstracts:
            source_value = "neighbourhoods"
        elif GT_abstract in tessellation_GT_abstracts:
            source_value = "tessellation"
        else:
            raise ValueError("Graph not found in either neighbourhood or tessellation lists!")

        # Update edges with the "source" attribute
        for edge in GT_abstract.es:
            edge["source"] = source_value
    
    if debug:
        for edge in GT_abstracts[0].es:
            print(edge.attributes())


    # Routing and creating GT for each prune_quantile
    GT_indices = set()

    # set an interator to do processing per GT_abstract in GT_abstracts
    GT_abstracts_iterator = iter(GT_abstracts)

    ##################################TEMPORARY CODE FOR DEBUGGING PURPOSES##################################
    def double_prune_quantiles(prune_quantiles):
        # Initialize an empty list to hold the new quantiles
        doubled_quantiles = []
        
        # Iterate through the original quantiles and insert a midpoint between consecutive values
        for i in range(len(prune_quantiles) - 1):
            doubled_quantiles.append(prune_quantiles[i])  # Append the current quantile
            # Calculate and append the midpoint between consecutive quantiles
            midpoint = (prune_quantiles[i] + prune_quantiles[i + 1]) / 2
            doubled_quantiles.append(midpoint)
        
        # Append the last element of the original list
        doubled_quantiles.append(prune_quantiles[-1])
        
        return doubled_quantiles
    ###########################################################################################################

    
    dubs = double_prune_quantiles(prune_quantiles)

    for prune_quantile in tqdm(dubs, desc="Routing", leave=False):
        GT_abstract = next(GT_abstracts_iterator)
        routenodepairs = { (e.source_vertex["id"], e.target_vertex["id"]): e["weight"] for e in GT_abstract.es }
        routenodepairs = sorted(routenodepairs.items(), key=lambda x: x[1])

        for poipair, poipair_distance in routenodepairs:
            poipair_ind = (G.vs.find(id=poipair[0]).index, G.vs.find(id=poipair[1]).index)
            
            if any(poi[0] == poipair[0] and poi[1] == 'neighbourhood' for poi in pois):
                # Neighbourhood routing logic
                neighbourhood_a = all_centroids.loc[all_centroids['nearest_node'] == poipair[0], 'neighbourhood_id'].values[0]
                neighbourhood_b = all_centroids.loc[all_centroids['nearest_node'] == poipair[1], 'neighbourhood_id'].values[0]
                exit_points_a = exit_points[exit_points['neighbourhood_id'] == neighbourhood_a].index
                exit_points_b = exit_points[exit_points['neighbourhood_id'] == neighbourhood_b].index

                shortest_path_length, best_path = float('inf'), None
                for ea in exit_points_a:
                    for eb in exit_points_b:
                        pair_id = tuple(sorted((ea, eb)))
                        if pair_id in processed_pairs: 
                            continue
                        processed_pairs.add(pair_id)
                        ea_vertex_index = G.vs.find(id=ea).index
                        eb_vertex_index = G.vs.find(id=eb).index
                        sp = G.get_shortest_paths(ea_vertex_index, eb_vertex_index, weights="weight", output="vpath")[0]
                        if sp and len(sp) < shortest_path_length:
                            shortest_path_length, best_path = len(sp), sp

                if best_path:
                    all_shortest_paths.append((poipair[0], poipair[1], shortest_path_length, best_path))
                    GT_indices.update(best_path)
                    
                    # Plotting for neighbourhood routing
                    GT = G.induced_subgraph([G.vs[idx] for idx in best_path])
                    GTs.append(GT)
            else:
                # Routing for tessellation pois (more general case)
                sp = set(G.get_shortest_paths(poipair_ind[0], poipair_ind[1], weights="weight", output="vpath")[0])
                GT_indices = GT_indices.union(sp)

         
        # Final step: Create the final GT from the indices we've accumulated
        GT = G.induced_subgraph(GT_indices)
        GTs.append(GT)


        if debug:
            # Plot the current GT
            GT_gjson = ig_to_geojson(GT)
            GT_geometries = [shape(geometry) for geometry in GT_gjson["geometries"]]
            GT_gdf = gpd.GeoDataFrame(geometry=GT_geometries, crs="EPSG:4326")

            # Plotting the GTs after each loop
            fig, ax = plt.subplots(figsize=(10, 10))
            GT_gdf.plot(ax=ax, color="blue", linewidth=1.5, label=f"GT for prune_quantile {prune_quantile}")
            
            all_centroids.plot(ax=ax, color="green", marker="o", markersize=50, label="All Centroids")
            exit_points.plot(ax=ax, color="red", marker="x", markersize=30, label="All Exit Points")

            plt.legend()
            plt.xlabel("Longitude")
            plt.ylabel("Latitude")
            plt.title(f"Greedy Triangulation (GT) for prune_quantile = {prune_quantile}")
            plt.show()

    return GTs, GT_abstracts


In [ ]:
## we need to load data again to ensure we take in neighbourhoods correctly
# Load all carall graphs in OSMNX format
G_caralls = {}
G_caralls_simplified = {}
locations = {}
parameterinfo = osmnxparameters['carall']

for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
    print(placeid + ": Loading location polygon and carall graph")
    
    if placeinfo["nominatimstring"] != '':
        location = ox.geocoder.geocode_to_gdf(placeinfo["nominatimstring"])
        if location.geometry[0].geom_type == 'MultiPolygon':
            location = location.explode(index_parts=False).reset_index(drop=True)
        location = fill_holes(extract_relevant_polygon(placeid, shapely.geometry.shape(location['geometry'][0])))
    else:
        # https://gis.stackexchange.com/questions/113799/how-to-read-a-shapefile-in-python
        shp = fiona.open(PATH["data"] + placeid + "/" + placeid + ".shp")
        first = next(iter(shp))
        try:
            location = Polygon(shapely.geometry.shape(first['geometry'])) # If shape file is given as linestring
        except:
            location = shapely.geometry.shape(first['geometry'])
    locations[placeid] = location
    
    G_caralls[placeid] = csv_to_ox(PATH["data"] + placeid + "/", placeid, 'carall')
    G_caralls[placeid].graph["crs"] = 'epsg:4326' # needed for OSMNX's graph_to_gdfs in utils_graph.py
    G_caralls_simplified[placeid] = csv_to_ox(PATH["data"] + placeid + "/", placeid, 'carall_simplified')
    G_caralls_simplified[placeid].graph["crs"] = 'epsg:4326' # needed for OSMNX's graph_to_gdfs in utils_graph.py

    print(placeid + ": loading and moving POIs")
    # Get the carall graph and location geometry
    location = locations[placeid]
    G_carall = G_caralls_simplified[placeid]

    # Load neighbourhoods and create GeoDataFrame for centroids
    neighbourhoods = load_neighbourhoods(PATH["data"] + placeid + "/")
    all_centroids = gpd.GeoDataFrame(columns=['neighbourhood_id', 'geometry'], crs='EPSG:4326')  
    exit_points = get_exit_nodes(neighbourhoods, G_carall) # requires osmnx G_carall, not igraph G_carall
        
    
    unique_id = 0  # Counter for unique IDs across neighbourhoods

    for name, gdf in neighbourhoods.items(): # Process each neighbourhood GeoDataFrame to get centroids, exit points, and neighbourhood IDs
        if gdf.empty:
            print(f"Warning: The GeoDataFrame for {name} is empty. Skipping...")
            continue
        print(f"Processing neighbourhoods in: {name}")

        # Assign a unique ID to each neighborhood in the GeoDataFrame to referance throughout
        gdf['neighbourhood_id'] = range(unique_id, unique_id + len(gdf))
        if debug == True:
            print(f"Assigned neighbourhood_ids from {unique_id} to {unique_id + len(gdf) - 1} for {name}")

        # Get centroids to inherit 'neighbourhood_id'
        centroids_gdf = get_neighbourhood_centroids(gdf)
        all_centroids = pd.concat([all_centroids, centroids_gdf], ignore_index=True)
        unique_id += len(gdf)  # Increment by the number of neighborhoods processed

    # Snap centroids to the closest nodes in the street network
    neighbourhood_nnids = set()
    for g in all_centroids['geometry']:
        n = ox.distance.nearest_nodes(G_carall, g.x, g.y)
        if n not in neighbourhood_nnids and haversine((g.y, g.x), (G_carall.nodes[n]["y"], G_carall.nodes[n]["x"]), unit="m") <= snapthreshold:
            neighbourhood_nnids.add(n)
        # Add nearest_node column to all_centroids by finding the nearest node for each centroid geometry
    all_centroids['nearest_node'] = all_centroids['geometry'].apply(
        lambda g: ox.distance.nearest_nodes(G_carall, g.x, g.y)) # we now have all_centroids with 'neighbourhood_id', 'geometry', 'nearest_node' columns

    # generate connections
    print("Loaded neighbourhoods and exit points", placeid + ": Generating networks")

    # Load POIs
    with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidsbikeall.csv') as f:
        tessellation_nnids = [int(line.rstrip()) for line in f]
    print("Loaded tesselation points")

    # combine nnids
    nnids = neighbourhood_nnids | set(tessellation_nnids)

    # Load networks
    #G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall', weighting=weighting)
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrackcarall', weighting=weighting) # to load car and cycle network

    # Generation
    (GTs, GT_abstracts) = greedy_triangulation_routing_mix(G_carall, neighbourhood_nnids, tessellation_nnids, weighting, prune_quantiles, prune_measure)
    (MST, MST_abstract) = mst_routing(G_carall, nnids, weighting)
    
    # Restore orignal edge lengths
    if weighting == True:
        restore_original_lengths(G_carall)
        for GT in GTs:
            restore_original_lengths(GT)
        restore_original_lengths(MST)


    # Write results
    results = {"placeid": placeid, "prune_measure": prune_measure, "poi_source": poi_source, "prune_quantiles": prune_quantiles, "GTs": GTs, "GT_abstracts": GT_abstracts, "MST": MST, "MST_abstract": MST_abstract}
    write_result(results, "pickle", placeid, poi_source, prune_measure, ".pickle", weighting=weighting)

In [136]:
debug = True

In [ ]:
len(tessellation_nnids)
len(neighbourhood_nnids)

In [ ]:
Audio(sound_file, autoplay=True)